## Generate news Articles

In [2]:
from app.llm_prompts import *
update_news_articles(prod_session)

In [1]:
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from dbb.models import Users, Lessons, UserAnswers , UseCases, NewsArticle # Adjust based on your actual models
import json

# Connect to the production database
prod_engine = create_engine(os.environ['DATABASE_URI'])
Session = sessionmaker(bind=prod_engine)
prod_session = Session()

# Function to copy data from production to test database
def copy_data_from_prod_to_test():
    # Copy Users
    prod_users = prod_session.query(Users).all()
    for user in prod_users:
        db.session.add(Users(id=user.id, username=user.username, email=user.email,
                            hashed_password=user.hashed_password, use_case_difficulty=1,
                            score=user.score if user.score else 0, is_admin=user.is_admin, lesson_id=user.lesson_id if user.lesson_id else 1))

    # Copy lessons
    prod_lessons = prod_session.query(Lessons).all()
    for lesson in prod_lessons:
        db.session.add(Lessons(id=lesson.id, title=lesson.title))

    # Copy user answers
    prod_user_answers = prod_session.query(UserAnswers).all()
    for user_answer in prod_user_answers:
        db.session.add(UserAnswers(id=user_answer.id, user_id=user_answer.user_id,
                       use_case_id=user_answer.use_case_id, question_id=user_answer.question_id,
                       option_id=user_answer.option_id, is_correct=user_answer.is_correct, created_at=user_answer.created_at,
                       lesson_id=user_answer.lesson_id
        ))

    # Copy use cases
    prod_use_cases = prod_session.query(UseCases).all()
    for use_case in prod_use_cases:
        db.session.add(UseCases(id=use_case.id, description=use_case.description,
                       type=use_case.type,
                        lesson_id=use_case.lesson_id,
                       created_by_user=use_case.created_by_user, risk_factors=json.dumps(use_case.risk_factors)
                       , difficulty_id=use_case.difficulty_id, news_article=use_case.news_article
        ))

    db.session.commit()

In [ ]:
import config
from datetime import datetime
from typing import Any, Dict, Optional

import markdown
from flask import Flask
from flask import current_app as app
from flask import jsonify, redirect, render_template, request, session, url_for
from flask_login import (LoginManager, UserMixin, current_user, login_required,
                         login_user, logout_user)

from app import db, create_app
from dbb.models import (DifficultyLevel, Lessons, Options, Questions, UseCases,
                        UserAnswers, UserLessonInteraction, Users)
from config import TestingConfig, ProductionConfig

# app = create_app('TestingConfig')
# app_context = app.app_context()
# app_context.push()
# db.create_all()

# # Populate the test database
# copy_data_from_prod_to_test()

In [ ]:
def find_similar_use_case(current_use_case_id, user_id):
    """
    Finds a use case similar to the current one based on shared risk factors.

    This function searches for a use case that shares risk factors with the current use case but has not yet been successfully completed by the user. This can help in recommending similar scenarios for further training or assessment.

    Args:
        current_use_case_id (int): The ID of the current use case.
        user_id (int): The ID of the user for whom similar use cases are being searched.

    Returns:
        UseCases | None: Returns a similar UseCase object if found, otherwise None.
    """
    current_use_case = UseCases.query.get(current_use_case_id)
    if not current_use_case:
        return None

    current_risk_factors = current_use_case.risk_factors

    # Query to find a similar use case based on risk factors
    similar_use_case = UseCases.query.filter(
        UseCases.id != current_use_case_id,
        UseCases.risk_factors.contains(current_risk_factors),
        ~UseCases.id.in_(
            db.session.query(UserAnswers.use_case_id)
            .filter(UserAnswers.user_id == user_id, UserAnswers.is_correct)
            .distinct()
        ),
    ).first()

    return similar_use_case

In [ ]:
current_use_case_id = 1
current_use_case = UseCases.query.get(current_use_case_id)
current_risk_factors = current_use_case.risk_factors
current_risk_factors


use_1  = json.loads(current_risk_factors)

In [ ]:
use_10 = json.loads(current_risk_factors)

In [ ]:
similar_use_cases = UseCases.query.filter(
    UseCases.id != current_use_case_id,
    ~UseCases.id.in_(
        db.session.query(UserAnswers.use_case_id)
        .filter(UserAnswers.user_id == 6, UserAnswers.is_correct)
        .distinct()
    ),
).all()

In [ ]:
def json_contains(json_obj, query, threshold=0.75):
    """Check if json_obj contains at least the threshold percentage of key-value pairs in query."""
    # Flatten the JSON objects for easier comparison
    def flatten_json(y):
        out = {}
        def flatten(x, name=''):
            if type(x) is dict:
                for a in x:
                    flatten(x[a], name + a + '.')
            else:
                out[name[:-1]] = x
        flatten(y)
        return out

    flat_json_obj = flatten_json(json_obj)
    flat_query = flatten_json(query)

    total_keys = len(flat_query)
    matching_keys = sum(1 for key in flat_query if key in flat_json_obj and flat_json_obj[key] == flat_query[key])

    similarity = matching_keys / total_keys if total_keys > 0 else 0
    print(similarity)

    return similarity >= threshold



In [ ]:
next_lesson = Lessons.query.filter_by(id=1).first()

use_cases = UseCases.query.filter_by(lesson_id=next_lesson.id).all()

lesson_data = {
    "id": next_lesson.id,
    "title": next_lesson.title,
    "use_cases": [
        {
            "id": use_case.id,
            "description": use_case.description,
            # Include other relevant use case data
        }
        for use_case in use_cases
    ],
}

In [ ]:
next_lesson

In [ ]:
use_1

In [ ]:
db.session.remove()
db.drop_all()
app_context.pop()

In [ ]:
import config
from datetime import datetime
from typing import Any, Dict, Optional

import markdown
from flask import Flask
from flask import current_app as app
from flask import jsonify, redirect, render_template, request, session, url_for
from flask_login import (LoginManager, UserMixin, current_user, login_required,
                         login_user, logout_user)

from app import db, create_app
from dbb.models import (DifficultyLevel, Lessons, Options, Questions, UseCases,
                        UserAnswers, UserLessonInteraction, Users)
from config import DevelopmentConfig

app = create_app(DevelopmentConfig)

In [ ]:
UseCases.query.filter_by(lesson_id=lesson_id)

In [ ]:
import pandas as pd
import os
from xlsx2csv import Xlsx2csv

def convert_excel_to_csv(folder_path, output_folder):
    # List all Excel files in the folder
    excel_files = [file for file in os.listdir(folder_path) if file.endswith('.xlsx')]

    # Initialize an empty list to store CSV file paths
    csv_file_paths = []

    # Loop through the list of Excel files
    for file in excel_files:
        try:
            # Construct full file path for the Excel file
            excel_file_path = os.path.join(folder_path, file)
            # Define the CSV output file path
            csv_file_path = os.path.join(output_folder, file.replace('.xlsx', '.csv'))
            # Convert Excel file to CSV
            Xlsx2csv(excel_file_path, outputencoding="utf-8").convert(csv_file_path)
            csv_file_paths.append(csv_file_path)
            print(f'Successfully converted {file} to CSV')
        except Exception as e:
            print(f'Failed to convert {file} to CSV: {e}')

    return csv_file_paths

def read_and_combine_csv(csv_file_paths):
    # Initialize an empty list to store DataFrames
    dataframes = []

    # Loop through the list of CSV file paths
    for csv_file in csv_file_paths:
        try:
            # Read the CSV file and append the resulting DataFrame to the list
            df = pd.read_csv(csv_file, skiprows=1)
            dataframes.append(df)
            print(f'Successfully read {csv_file}')
        except Exception as e:
            print(f'Failed to read {csv_file}: {e}')

    # Combine all DataFrames into one
    if dataframes:
        combined_df = pd.concat(dataframes, ignore_index=True)
        return combined_df
    else:
        return pd.DataFrame()  # Return an empty DataFrame if no files were read

# Usage
folder_path = '/Users/lucavehbiu/Documents/GitHub/investigamer-cfa/likuidimet'
output_folder = '/Users/lucavehbiu/Documents/GitHub/investigamer-cfa/likuidimet_csv'
csv_files = convert_excel_to_csv(folder_path, output_folder)
combined_dataframe = read_and_combine_csv(csv_files)
print(combined_dataframe)

In [ ]:
combined_dataframe

In [ ]:
delivered_df = combined_dataframe[combined_dataframe['Status'] == 'Delivered'].copy()

print(len(delivered_df))

delivered_df_no_dupli = delivered_df.drop_duplicates().copy()

print(len(delivered_df_no_dupli))

delivered_df_no_dupli

In [ ]:
delivered_df_no_dupli['vlera_pakos'] = delivered_df_no_dupli['Pay on delivery'].str.replace(' ALL', '').astype(float)
delivered_df_no_dupli['transport'] = delivered_df_no_dupli['Transport'].str.replace(' ALL', '').astype(float)

delivered_df_no_dupli['Updated'] = pd.to_datetime(delivered_df_no_dupli['Updated'], format='%d %B %Y %H:%M')
# Format the datetime objects to 'YYYY-MM'
delivered_df_no_dupli['Month_Year'] = delivered_df_no_dupli['Updated'].dt.strftime('%Y-%m')

In [ ]:

delivered_df_no_dupli['transport_287_leke'] = 287.5

delivered_df_no_dupli['differenca'] =  delivered_df_no_dupli['transport'] - delivered_df_no_dupli['transport_287_leke']




In [ ]:
delivered_df_no_dupli['receiver'] =  delivered_df_no_dupli['Receiver'].str.lower()

dddelivered_df_no_dupli_no_kosove = delivered_df_no_dupli[~delivered_df_no_dupli['receiver'].str.contains('prishtine|prizren|ferizaj|peje|viti|dragash|prishtina|mitrovice|vushtri|decan|sugareke|suhareke|prizren|shtime|malisheve|rahovec|gjakove|fushe kosove|Ferizaj')].copy()




In [ ]:
# Define the bins
bins = [-float('inf'), 0.1, 100, 200, 300, 500, 1000, 2000, float('inf')]
# Define the labels for the bins
labels = [
    '0.0 leke', '0.0 to 100 leke', '101 to 200 leke', '201 to 300 leke',
    '301 to 500 leke', '501 to 1000 leke', '1001 to 2000 leke', 'Over 2000 leke'
]

dddelivered_df_no_dupli_no_kosove['categori_diference'] = pd.cut(dddelivered_df_no_dupli_no_kosove['differenca'], bins=bins, labels=labels, right=False)

dddelivered_df_no_dupli_no_kosove


In [ ]:
df = dddelivered_df_no_dupli_no_kosove.groupby(['Month_Year', 'categori_diference']).aggregate({'vlera_pakos' : 'sum',
                                                    'transport': 'sum',
                                                    'Barcode': 'nunique',
                                                    'Receiver':'nunique'
                                                       }).reset_index()

df['nr_transport_total_muaj'] = df[['Month_Year', 'Barcode']].groupby('Month_Year').transform('sum')['Barcode']
df['perqindje_e_porosive'] = df['Barcode'] / df['nr_transport_total_muaj']
df.sort_values(['categori_diference', 'Month_Year'])

In [ ]:
df = dddelivered_df_no_dupli_no_kosove.groupby('Month_Year').aggregate({'vlera_pakos' : 'sum',
                                                    'transport': 'sum',
                                                    'Barcode': 'nunique',
                                                    'Receiver':'nunique'
                                                       }).reset_index()



df['transport_287_leke'] = 287.5 * df['Barcode']
df['differenca_transport_aktual_transport_287_leke'] = df['transport'] - df['transport_287_leke']
df.columns = ['muaji', 'vlera_totale_e_pakove', 'vlera_totale_transport', 'nr_barcode', 'nr_receiver', 'vlera_total_transport_288_leke', 'differenca_transport_aktual_transport_287_leke_all']
df

In [ ]:
a = df[df['categori_diference'] == '0.0 leke']

a['perqindje_e_porosive'] = (a['perqindje_e_porosive'].round(3) * 100).astype(str) + '%'

a.columns = ['muaji', 'categori_diference', 'vlera_totale_e_pakove', 'vlera_totale_transport', 'nr_barcode', 'nr_receiver', 'nr_barcode_muaji', 'perqindja']

a

In [ ]:
df.tail(50)

In [ ]:
df.to_excel('~/Downloads/differenca_transport_per_kategori.xlsx')

In [ ]:
dddelivered_df_no_dupli_no_kosove.to_excel('~/Downloads/raw_data_me_kategori_diference.xlsx')

In [ ]:
dddelivered_df_no_dupli_no_kosove